In [2]:
# !pip install -q huggingface_hub
# !pip install -q -U trl transformers accelerate peft
# !pip install -q -U datasets bitsandbytes einops wandb
print("Start finetuning...")

Start finetuning...


In [3]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import wandb
wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1nv1nce (invince). Use `wandb login --relogin` to force relogin


In [6]:
from datasets import load_dataset
import torch,einops
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [7]:
dataset = load_dataset("json",data_files="/root/autodl-tmp/processdata/PostCoR.json", split="train")
dataset = dataset.select(range(2500))

In [8]:
base_model_name ="/root/autodl-tmp/Llama-2-13b-chat-hf"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,#在4bit上，进行量化
#     bnb_4bit_use_double_quant=True,# 嵌套量化，每个参数可以多节省0.4位
#     bnb_4bit_quant_type="nf4",#NF4（normalized float）或纯FP4量化 博客说推荐NF4
#     bnb_4bit_compute_dtype=torch.float16,
# )

In [9]:
device_map = "auto"

In [10]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,#本地模型名称
    # quantization_config=bnb_config,#上面本地模型的配置
    device_map=device_map,#使用GPU的编号
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/root/miniconda3/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
output_dir = "./results"
training_args = TrainingArguments(
    report_to="wandb",
    output_dir=output_dir,#训练后输出目录
    per_device_train_batch_size=4,#每个GPU的批处理数据量
    gradient_accumulation_steps=8,#在执行反向传播/更新过程之前，要累积其梯度的更新步骤数
    learning_rate=2e-4,#超参、初始学习率。太大模型不稳定，太小则模型不能收敛
    logging_steps=10,#两个日志记录之间的更新步骤数
    # max_steps=200#要执行的训练步骤总数
    num_train_epochs = 2.0
)
max_seq_length = 1024
#TrainingArguments 的参数详解：https://blog.csdn.net/qq_33293040/article/details/117376382

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

In [14]:
trainer.train()

Step,Training Loss
10,1.416300
20,0.625100
30,0.315600
40,0.268100
50,0.249600
60,0.246700
70,0.241200
80,0.237100
90,0.228800
100,0.230900


TrainOutput(global_step=156, training_loss=0.3397178382445604, metrics={'train_runtime': 9547.8494, 'train_samples_per_second': 0.524, 'train_steps_per_second': 0.016, 'total_flos': 3.7577640654987264e+17, 'train_loss': 0.3397178382445604, 'epoch': 1.9968})

In [15]:
import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

/root/miniconda3/lib/python3.8/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/Llama-2-13b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(
